In [1]:
import openpyxl
from openpyxl.styles import Font
from selenium import webdriver
from selenium.webdriver.common.by import By
# https://velog.io/@sangyeon217/deprecation-warning-executablepath-has-been-deprecated
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("headless")
options.add_argument("--window-size=1920,1080")
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
options.add_argument(f'user-agent={user_agent}')
#NOTE https://stackoverflow.com/questions/47316810/unable-to-locate-elements-on-webpage-with-headless-chrome


In [68]:
filename = "files/XLM 완료 Bbrick_취합파일 정리_2200617(원태님).xlsx"
workbook = openpyxl.load_workbook(filename=filename, data_only=True)
sheet = workbook["입금"]

In [71]:
explorer_url_hash = {
    'ETH': [ ['ETH', "ETC"], "https://etherscan.io/tx/", "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]"]
    ,'BSC': [ ['BSC'], "https://bscscan.com/tx/", "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]" ]
    ,'TRX': [ ["TRX", "TRON", "TRC20USDT"], "https://tronscan.org/#/transaction/" , "//*[@id='root']/div[2]/main/div/div/div[3]/div[2]/div/div/div/table/tbody/tr[1]/td/span/div/span/div/div/span/div/a/div"]
    ,"HECO": [ ["HECO", "HRC"], "https://hecoinfo.com/tx/", "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]" ]
    ,"MATIC": [ ["MATIC"], "https://polygonscan.com/tx/", "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]" ]
    ,"ETC": [ ["ETC"], "https://etcblockexplorer.com/tx/", "//*[@id='txSpecific']/span[19]" ]
    ,"KLAY": [ ["KLAY"], "https://scope.klaytn.com/tx/", "//*[@id='root']/div/div[2]/div[2]/div/div/div/div/div/div[1]/div[3]/div[2]/div/span[1]/div/a" ]
    ,"BTC": [ ["KLAY"], "https://blockstream.info/tx/", "//*[@id='root']/div/div[2]/div[2]/div/div/div/div/div/div[1]/div[3]/div[2]/div/span[1]/div/a" ]
}


cell_color = Font(name="Arial", color="8f34eb")

# 둘째자리에서 첫째자리로 반올림
value = 287.1260113
# 다 돌아가면서, value : from_address 맵을 만들어야겠다




lists = driver.find_element(by=By.XPATH, 
value="//*[@id='transaction-box']/div[2]/div[3]")
lists.text

import math
result = lists.text.split("\n")

value = 2.9996
# 파이썬 반올림
for i, each in enumerate(result):
    if i % 2 == 1:
        BTC_AMOUNT = result[i]
        AMOUNT, BTC = BTC_AMOUNT.split()
        AMOUNT = float(AMOUNT)

        if round(value, 3) == round(AMOUNT, 3):
            print(result[i-1])



3LLRS688CkBg71m4qvEffbHh9sMQZfH5or


In [9]:
explorer_url_hash = {
    'ETH': [ ['ETH', "ETC"], "https://etherscan.io/tx/", "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]"]
    ,'BSC': [ ['BSC'], "https://bscscan.com/tx/", "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]" ]
    ,'TRX': [ ["TRX", "TRON", "TRC20USDT"], "https://tronscan.org/#/transaction/" , "//*[@id='root']/div[2]/main/div/div/div[3]/div[2]/div/div/div/table/tbody/tr[1]/td/span/div/span/div/div/span/div/a/div"]
    ,"HECO": [ ["HECO", "HRC"], "https://hecoinfo.com/tx/", "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]" ]
    ,"MATIC": [ ["MATIC"], "https://polygonscan.com/tx/", "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]" ]
    ,"ETC": [ ["ETC"], "https://etcblockexplorer.com/tx/", "//*[@id='txSpecific']/span[19]" ]
    ,"KLAY": [ ["KLAY"], "https://scope.klaytn.com/tx/", "//*[@id='root']/div/div[2]/div[2]/div/div/div/div/div/div[1]/div[3]/div[2]/div/span[1]/div/a" ]
}

cell_color = Font(name="Arial", color="ad03fc")


def get_addr_of_network(target_network, start):
    num_written = 0
    start_time = time.time()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    global sheet
    for i in range(start, sheet.max_row + 1):
        network = sheet[f"L{str(i)}"].value
        if network in explorer_url_hash[target_network][0]:
            tx_hash = sheet[f"S{str(i)}"].value
            if "Internal" in tx_hash or sheet[f"N{str(i)}"].value is not None: #요게 잘못됨, 무조건 continue하게 되어있음
                continue 
            tx_hash = tx_hash if tx_hash[:2] == "0x" else "0x" + tx_hash
            # driver = webdriver.Chrome("chromedriver.exe", options=options) 
            driver.get(  explorer_url_hash[target_network][1] + tx_hash)
            driver.implicitly_wait(2)
            try:
                element = driver.find_element(by=By.XPATH, value=explorer_url_hash[target_network][2])
                from_address = element.text
                sheet[f"N{str(i)}"].value = from_address
                print("row:", i, "from_address:", from_address, "txhash:", tx_hash)
                num_written += 1    
            except:
                sheet[f"N{str(i)}"].value = "Wrong chain or invalid existing tx_hash"
                print("Wrong chain or invalid existing tx_hash")

            sheet[f"N{str(i)}"].font = cell_color
            # driver.quit()
            
    print(f"written {num_written}, took {time.time() - start_time} seconds")
    driver.quit()

In [44]:
explorer_url_hash = {
    "ATOM": [ ["ATOM"], "https://atomscan.com/transactions/", 
    "//*[@id='app']/div[2]/div/div/div[2]/div/div[3]/div/article/div[2]/div[1]/div[2]/span/span/a/span",
    "//*[@id='app']/div[2]/div/div/div[2]/div/div[2]/div/div/div/div[6]/div[2]",
    False ],
    "EOS": [ ["EOS"], "https://eosflare.io/tx/", 
    "//*[@id='actions']/div[1]/div[3]/div/div[1]/a[1]",
    "//*[@id='actions']/div[1]/div[3]/div/div[2]",
    False ],
    "TERRA": [ ["TERRA"], "https://finder.terra.money/classic/tx/", 
    "//*[@id='root']/section/section/div[3]/div[2]/section[3]/div/a",
    "//*[@id='root']/section/section/div[2]/div[6]/div[2]",
    False ],
    "XRP": [ ["XRP"], "https://xrpscan.com/tx/",
    "//*[@id='root']/div/div/div/div/div[3]/div/div[1]/div[1]/div[2]/div[1]/div/table/tbody/tr[3]/td[2]/a",
    "//*[@id='root']/div/div/div/div/div[3]/div/div[1]/div[1]/div[2]/div[1]/div/table/tbody/tr[4]/td[2]/span/span/span",
    False ],
    "XLM": [ ["XLM"], "https://steexp.com/tx/",
    "//*[@id='operation-table']/tbody/tr/td[1]/span/a",
    "//*[@id='main-content']/div/div[1]/div/div[2]/table/tbody/tr[4]/td[2]",
    False ]
}




cell_color = Font(name="Arial", color="8f34eb")


def get_addr_memo_of_network(target_network, start):
    num_written = 0
    start_time = time.time()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    global sheet
    for i in range(start, sheet.max_row + 1):
        network = sheet[f"L{str(i)}"].value
        if network in explorer_url_hash[target_network][0]:
            tx_hash = sheet[f"S{str(i)}"].value
            if "Internal" in tx_hash or sheet[f"N{str(i)}"].value is not None: #요게 잘못됨, 무조건 continue하게 되어있음
                continue
            # invalid할 때

            if explorer_url_hash[target_network][4]:
                tx_hash = tx_hash if tx_hash[:2] == "0x" else "0x" + tx_hash
            else:
                pass
            # driver = webdriver.Chrome("chromedriver.exe", options=options) 
            driver.get(  explorer_url_hash[target_network][1] + tx_hash)
            driver.implicitly_wait(10)
            try:
                from_address_element = driver.find_element(by=By.XPATH, value=explorer_url_hash[target_network][2])
                driver.implicitly_wait(10)
                from_address = from_address_element.get_attribute("href").split("/")[-1]
                sheet[f"N{str(i)}"].value = from_address
                memo_element = driver.find_element(by=By.XPATH, value=explorer_url_hash[target_network][3])
                driver.implicitly_wait(10)
                memo = memo_element.text
                print("row:", i, "from_address:", from_address, "memo:", memo, "txhash:", tx_hash)
                # None이면...
                sheet[f"O{str(i)}"].value = memo
    
                num_written += 1    
            except:
                # sheet[f"N{str(i)}"].value = ""
                print("Wrong chain or invalid existing tx_hash")

            sheet[f"N{str(i)}"].font = cell_color
            sheet[f"O{str(i)}"].font = cell_color
            driver.implicitly_wait(5)
            
    driver.quit()
    print(f"written {num_written}, took {time.time() - start_time} seconds")

In [19]:
tx_hash = "2B8C6B92924225EB364BF5062B78D32568044957079778698743D07157CCCDD3"
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(3)
time.sleep(1)
driver.get(f"https://atomscan.com/transactions/{tx_hash}")
driver.implicitly_wait(3)
time.sleep(2)
driver.save_screenshot('아.png')

element = driver.find_element(by=By.XPATH, 
value="//*[@id='app']/div[2]/div/div/div[2]/div/div[3]/div/article/div[2]/div[1]/div[2]/span/span/a/span")
time.sleep(2)
driver.implicitly_wait(3)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache


In [22]:
print(element.text)
memo = driver.find_element(by=By.XPATH, 
value="//*[@id='app']/div[2]/div/div/div[2]/div/div[2]/div/div/div/div[6]/div[2]")
time.sleep(2)
print(memo.text)

cosmos1yp8q44e4t4evksnczmzlw0ngxj4hwuv5rjw3a8
<No Memo>


In [16]:
element.text.split('\n')[1] == '1'

IndexError: list index out of range

In [14]:
element.text.split('\n')[1] == '1'

True

In [45]:
# result = element.text.split("\n")
element.text
print(result)

['pzyhz3vr8gr3kajw9srx...ej83tm8v6ces4nu9rgzr', '1.06859436', 'qrl59ue7seauw2lsj370...70gq04zwsh5unkfg6scm', '4.59000000']


In [36]:
current = time.time()
from time import strftime, localtime

strftime("%Y-%m-%d %I:%M:%S", localtime(current))

'2022-06-22 01:14:36'

In [22]:
value = 1286.76892314
# 파이썬 반올림
for i in range(len(result)):
    if i % 3 == 1:
        BCC_AMOUNT = result[i]
        AMOUNT, BCC = BCC_AMOUNT.split()
        AMOUNT = float(AMOUNT.replace(",", ""))

        if round(value, 3) == round(AMOUNT, 3):
            print(result[i-1])
    

qr3ym377d5w8fh72g8lvnyg674v9q0kqj55srj09p8


In [23]:
# 함수를 인자로
def other_method(a):
    print(a)

val = other_method
val(123)


123


In [25]:
def preprocessing_func(AMOUNT):
    AMOUNT = float(AMOUNT.replace(",", ""))
    return AMOUNT

def _get_matched_from_address(STEP, QUANTITY, result, preprocessing_func = None):
    for i in range(len(result)):
        if i % STEP == 1:
            TOKEN_AMOUNT = result[i]
            AMOUNT, _ = TOKEN_AMOUNT.split()
        
            AMOUNT = preprocessing_func(AMOUNT)
            if round(QUANTITY, 3) == round(AMOUNT, 3):
                return result[i-1]

answer = _get_matched_from_address(3, value, result, preprocessing_func)
answer

'qr3ym377d5w8fh72g8lvnyg674v9q0kqj55srj09p8'

In [45]:

get_addr_memo_of_network("XLM", 107)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


row: 116 from_address: GC4EWDSIAH6IG24OWCFVR44LVHIPKX7ZBHQ26IGKTNZ2QKNRPSFOPJPI memo: 183316 txhash: 76720076c8fadf18992ef18edaeb13b64343309ee762cdfb551c8d9ccc616073
row: 169 from_address: GC4EWDSIAH6IG24OWCFVR44LVHIPKX7ZBHQ26IGKTNZ2QKNRPSFOPJPI memo: 183316 txhash: a16a4d39a15e9080326560fd8d5bc0072384080ffdedeb81d4d44cb177cebbe5
row: 189 from_address: GC4EWDSIAH6IG24OWCFVR44LVHIPKX7ZBHQ26IGKTNZ2QKNRPSFOPJPI memo: 183316 txhash: ab904c8ad9256bbb1c60934546b1df83e2c617142ab1da35c1cb388799448c65
row: 206 from_address: GC4EWDSIAH6IG24OWCFVR44LVHIPKX7ZBHQ26IGKTNZ2QKNRPSFOPJPI memo: 183316 txhash: 10b565e648de6e697baa8e8713d809e0f527bcb0ef6b92ebeaed3bf5171fe4ce
row: 219 from_address: GC4EWDSIAH6IG24OWCFVR44LVHIPKX7ZBHQ26IGKTNZ2QKNRPSFOPJPI memo: 183316 txhash: afec6cbab3ecf893e0ab247bbe2fefa781ef2c4ce99c2722b3d49a0eb856d0b6
row: 247 from_address: GC4EWDSIAH6IG24OWCFVR44LVHIPKX7ZBHQ26IGKTNZ2QKNRPSFOPJPI memo: 183316 txhash: 24292b9e5df31c25134247708dfb5e972a2e9e16f58aad1af79c19b070d3bad3
row:

In [18]:

get_addr_memo_of_network("XRP", 345)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


row: 345 from_address: rLcxBUrZESqHnruY4fX7GQthRjDCDSAWia (Klaytn XRP (KXRP)) memo: 3396311572 txhash: 98F559FF2256AD0B70FF9A812B4BD314E78D6433D2C900030E4809695066AB65
row: 437 from_address: rwSdwXsQVCjYXUyempsGxk1Sgws6perYZL (GOPAX) memo: 3396311572 txhash: CB95DBCA91633182857D734C943DD59FE1DF9B736365BE28FBAC3F680E53146B
row: 439 from_address: rwSdwXsQVCjYXUyempsGxk1Sgws6perYZL (GOPAX) memo: 3396311572 txhash: 82CBBE4DEE7C791D5BAC0B1858D1AE6949F2D685E13B80AA995FB177AE92ECEE
row: 440 from_address: rwSdwXsQVCjYXUyempsGxk1Sgws6perYZL (GOPAX) memo: 3396311572 txhash: 6DCAA3327548334E14417DB21FE6DE167918FB3F0CE3929D4A809A6BB83E4378
row: 451 from_address: rwSdwXsQVCjYXUyempsGxk1Sgws6perYZL (GOPAX) memo: 3396311572 txhash: BB2D2A0B2FB600842F90D21247182B4454B40F760050E5F0FAC29A6AFC3285D3
row: 452 from_address: rwSdwXsQVCjYXUyempsGxk1Sgws6perYZL (GOPAX) memo: 3396311572 txhash: 9FFEBA3AFF791C1A574115217F8B81257DD0EE5C4ACE4E07CE185DA2A0059E47
row: 485 from_address: rwSdwXsQVCjYXUyempsGxk1Sgws6p

In [86]:

get_addr_memo_of_network("TERRA", 29)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


row: 214 from_address: terra13yxhrk08qvdf5zdc9ss5mwsg5sf7zva9xrgwgc memo:  txhash: CB5B35EAAA54BA526D1B0C5EECDBBEB8400055F0884EF5FCBB2ED2D14F4E1A91
row: 216 from_address: terra13yxhrk08qvdf5zdc9ss5mwsg5sf7zva9xrgwgc memo:  txhash: D7037B78C2199F08722602D56F5527649E5F2776EC8A35CC3E579A69B260A44A
row: 221 from_address: terra13yxhrk08qvdf5zdc9ss5mwsg5sf7zva9xrgwgc memo:  txhash: 4F0B74313994A8B6795E55DA088A357F7B3271546CDBBECF4CE59EE6D8D99B03
row: 222 from_address: terra13yxhrk08qvdf5zdc9ss5mwsg5sf7zva9xrgwgc memo:  txhash: 78FB0C6192C56EE42697096A62B03C711132D69624F451DF0DC85740FBEF6E04
Wrong chain or invalid existing tx_hash
Wrong chain or invalid existing tx_hash
row: 238 from_address: terra13yxhrk08qvdf5zdc9ss5mwsg5sf7zva9xrgwgc memo:  txhash: D7037B78C2199F08722602D56F5527649E5F2776EC8A35CC3E579A69B260A44A
row: 241 from_address: terra13yxhrk08qvdf5zdc9ss5mwsg5sf7zva9xrgwgc memo:  txhash: F90E593629167EDC456BBED0E0E73A78C2B6D5831AFF4A55F97FBE6DD98C55B4
row: 242 from_address: terra13yx

In [77]:

get_addr_memo_of_network("EOS", 651)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


row: 651 from_address: gopaxdeposit memo: (Memo: 8115255) txhash: ff1e6263a76f76df4ed5125c7a346ee34a6b841e10d4980a2137ba4bcec12b39
Wrong chain or invalid existing tx_hash
row: 654 from_address: gopaxdeposit memo: (Memo: 8115255) txhash: 21fe9fe97ce34d780965c6d6d697f3a1d71d637877699d55c036f599c202ffc1
row: 715 from_address: gopaxdeposit memo: (Memo: 8115255) txhash: 61ee0064cb68d25154f9c5d3f8f2c462198610d2022c5c1967169dd90e51c6d7
Wrong chain or invalid existing tx_hash
Wrong chain or invalid existing tx_hash
row: 1494 from_address: gopaxdeposit memo: (Memo: 106375124) txhash: 4ac3b0ca5c6b546991b19f10886556d076c5cdfa1df242ba794ebb770b2a463e
row: 1590 from_address: gopaxdeposit memo: (Memo: 106375124) txhash: 4b755a6a6d710462917c9af7e423398e6956f4f7d91c01a3cf30b7b863b8ea95
row: 1684 from_address: gopaxdeposit memo: (Memo: 106375124) txhash: f47a132e0ef14f192764ad13accd4e729c80cd831c1bcad770d5aa3d3c950346
row: 1791 from_address: vuniyuoxoeub memo: (Memo: 106375124) txhash: 7fd98e8c7b2b4d6f

In [62]:
get_addr_of_network('KLAY', 3)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


row: 74 from_address: 0x980e9dfeaa8ea715e97827bb770fac8b97090ef1 txhash: 0x0501bbc6cc2a6151a02ed16ed71f98597df7f63ebe2ab801c27fb469a1831edb
row: 101 from_address: 0x980e9dfeaa8ea715e97827bb770fac8b97090ef1 txhash: 0xf9268b6fd65661090e207f880f55beca9e152bdce43c9e5ea80ec1c28185570b
Wrong chain or invalid existing tx_hash
Wrong chain or invalid existing tx_hash
Wrong chain or invalid existing tx_hash
Wrong chain or invalid existing tx_hash
row: 137 from_address: 0x980e9dfeaa8ea715e97827bb770fac8b97090ef1 txhash: 0xf61463a82ffacf34a266e3b4ba353a73f2e01e1445b9a7eb2ec261cdc4ecf8b3
row: 162 from_address: 0x980e9dfeaa8ea715e97827bb770fac8b97090ef1 txhash: 0x57703c6380e7825ebee8b8bd9c10b1dd9c1092c9e4022d5e6e5a3aa4018d65dd
row: 172 from_address: 0x980e9dfeaa8ea715e97827bb770fac8b97090ef1 txhash: 0xf58dbe5a7f457d2f325c14041ba655a11ab490b8df6601d3792debb7864f0527
row: 179 from_address: 0x980e9dfeaa8ea715e97827bb770fac8b97090ef1 txhash: 0x28a05324dbbc0f6d0fd495a6b8ff7b44428cea743de1ccc0926cdb7d5154

In [56]:
get_addr_of_network('ETC', 23)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


row: 23 from_address: "0x4A0fB8bbD83A7B6fdee2e31601C4E9eAf294c43b" txhash: 0x1af0c1f70aa0aab493c51111d00bb42f22e876932807c34f73162d02ea4d4281
row: 127 from_address: "0x4A0fB8bbD83A7B6fdee2e31601C4E9eAf294c43b" txhash: 0xe91d46191ea14a93cdea0eb030c7cfc4ce14484a896b8aeb1b4c8d93c7399636
row: 138 from_address: "0x4A0fB8bbD83A7B6fdee2e31601C4E9eAf294c43b" txhash: 0x589a8a06304a2042616551f5d16ed6af6d7a8c3d84ffef9494394bf9510022e6
row: 140 from_address: "0x4A0fB8bbD83A7B6fdee2e31601C4E9eAf294c43b" txhash: 0x23bb78408691fcaa27438c652af8d0469a2ce774ee227c3e8c3b3a5dadb18635
row: 145 from_address: "0x4A0fB8bbD83A7B6fdee2e31601C4E9eAf294c43b" txhash: 0x1e056cffa1e6c4b0b359f3fd591a385ec979b078201b208a8adf8307d53ad1e2
row: 154 from_address: "0x4A0fB8bbD83A7B6fdee2e31601C4E9eAf294c43b" txhash: 0xe254746004209b7771a7037749a0f95265eb5c796c9166068fd788bf3865e81f
row: 165 from_address: "0x4A0fB8bbD83A7B6fdee2e31601C4E9eAf294c43b" txhash: 0x058089e42fe72be0d80e6e86900ab883ac9a2b7127998d57a64cff8ef2fd937c


In [48]:
get_addr_of_network('MATIC', 83)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


written 0, took 1.2645759582519531 seconds


In [34]:
get_addr_of_network('HECO', 698)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


row: 699 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0x422e0a6f57c91316ec79c25d7e13f52143e0c60af21cc453165800f5f068e26b
row: 710 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0x75575522baf78d7775d70c29015f33d5b6e4ff26b1ef4cdcd285ed827c18d58b
row: 712 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0xafac8e08af3031078c06434734b7c1054c54b85876334a195f523f970d97bcee
row: 713 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0xdb4c1e82e0afd45bd6c72b223dac6ebcd699f5af6e7d7c3d30b3084e715f0e48
row: 720 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0xf1a4b6fdd24c232c6b7e7e46f40c3be00ace147f25b2047e0f154241064474ac
row: 721 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0x3dd74f547bdda61e930bf724a64286face9a9aef4a27e22e3e95423d40d8df7e
row: 731 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0x6928b6b73d382d380867c76cf378ac4c297cb338eafde5b9fe49aad4264c1ce4
row: 733 from

In [11]:
get_addr_of_network('BSC', 1400)

<ipython-input-10-3a66e9a4a5ae>:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=options)


row: 1400 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0xf845c02fc67ef74250d6986a7c8809107cb52e5af95f294884428248376adae7
row: 1403 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0xc821a87129b8c16d298e62c8ebc85fb03b7779403d7c64ed8416bf32bc6fb85d
row: 1405 from_address: 0xc8864df74a02b69992c4e680b50b9d77e13ee41f txhash: 0x687928d60cdc005e71bbdd2de53c5d74397fe3e2ab96a466c104a31fa1537e19
row: 1407 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0x66f7bedd16bc0edac8370f10a60a821ec1ab86569e475950ee5ef3dd5e872b6f
row: 1409 from_address: 0x41772edd47d9ddf9ef848cdb34fe76143908c7ad (FTX Exchange) txhash: 0x41edbf7fb370af52c56d3ca1e6881739cdf09d4c43bd65abd5a064667b10ad75
row: 1412 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0x9635d1f0ceb30ac58b400b3d1d9dae8cf1c5b857d51585212d3159a9779af5b3
row: 1414 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0xc60674f1544500e82cb8b2d4c3618b331a223bba7b75072d9fbcf821

In [45]:
explorer_url_hash = {
    'ETH': [ ['ETH', "ETC"], "https://etherscan.io/tx/", "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]"], 
    'BSC': [ ['BSC'], "https://bscscan.com/tx/", "//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]" ] ,
    'TRX': [ ["TRX", "TRON", "TRC20USDT"], "https://tronscan.org/#/transaction/" , "//*[@id='root']/div[2]/main/div/div/div[3]/div[2]/div/div/div/table/tbody/tr[1]/td/span/div/span/div/div/span/div/a/div"],
    # TRX가 콘트랙 아이디도 있고,,, 요소가 하나로 안 떨어지네. 시발
}


cell_color = Font(name="Arial", color="fc0339")

# 브라우저를 불러오는 속도가 느려서 못 잡아낸 것, 로직엔 잘못이 없다.
def get_addr_of_TRX(target_network, start):
    num_written = 0
    start_time = time.time()
    global sheet
    for i in range(start, sheet.max_row + 1):
        network = sheet[f"L{str(i)}"].value
        if network in explorer_url_hash[target_network][0]:
            tx_hash = sheet[f"S{str(i)}"].value
            if "Internal" in tx_hash or sheet[f"N{str(i)}"].value is not None: #요게 잘못됨, 무조건 continue하게 되어있음
                continue 
            # tx_hash = tx_hash if tx_hash[:2] == "0x" else "0x" + tx_hash
            # driver = webdriver.Chrome("chromedriver.exe", options=options) 
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
            driver.get(  explorer_url_hash[target_network][1] + tx_hash)
            driver.implicitly_wait(2.5)
            try:
                former = driver.find_element(by=By.XPATH, 
                value="/html/body/div[1]/div[2]/main/div/div/div[3]/div[2]/div/div/div/div/div[3]/div[2]/div/div/div/div[2]/div/span/div/div/span/div/a/div")
                latter = driver.find_element(by=By.XPATH, 
                value="/html/body/div[1]/div[2]/main/div/div/div[3]/div[2]/div/div/div/div/div[3]/div[2]/div/div/div/div[2]/div/span/div/div/span/div/div/span/span")

                from_address = former.text.replace("\n","")
                memo = latter.text.replace("\n","")

                result = from_address + " " + memo
                
                sheet[f"N{str(i)}"].value = result
                print("row:", i, "from_address:", from_address, "txhash:", tx_hash)
                num_written += 1    
            except:
                sheet[f"N{str(i)}"].value = "Wrong chain or invalid existing tx_hash"
                print("Wrong chain or invalid existing tx_hash")

            sheet[f"N{str(i)}"].font = cell_color
            driver.quit()
            
    print(f"written {num_written}, took {time.time() - start_time} seconds")

In [40]:
#NOTE Xpath: get rid of the text() method at the end
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
tx_hash = "76720076c8fadf18992ef18edaeb13b64343309ee762cdfb551c8d9ccc616073"
driver.get(f"https://steexp.com/tx/{tx_hash}")
driver.implicitly_wait(2)
former = driver.find_element(by=By.XPATH, 
value="//*[@id='operation-table']/tbody/tr/td[1]/span/a")
latter = driver.find_element(by=By.XPATH, 
value="//*[@id='main-content']/div/div[1]/div/div[2]/table/tbody/tr[4]/td[2]")




from_address = former.get_attribute("href").split("/")[-1]
memo = latter.text
print(from_address, memo)
result = from_address + " " + memo
print(result)
# print(latter.text)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


GC4EWDSIAH6IG24OWCFVR44LVHIPKX7ZBHQ26IGKTNZ2QKNRPSFOPJPI 183316
GC4EWDSIAH6IG24OWCFVR44LVHIPKX7ZBHQ26IGKTNZ2QKNRPSFOPJPI 183316


In [39]:
print(from_address, memo)

In [46]:
get_addr_of_TRX('TRX', 59)

written 0, took 0.014672994613647461 seconds


In [46]:
workbook.save("files/XLM 완료 Bbrick_취합파일 정리_2200617(원태님).xlsx")